In [12]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **Load the training data**

We load the training data.

In [13]:
import datetime
from implementations import *

data_load = load_data("x_train.csv")
pred = load_data("y_train.csv")
print(f"The data has {data_load.shape[0]} samples and {data_load.shape[1]} features !")

The data has 328135 samples and 321 features !


The first column is states and I create a new column to classify with the ranking of PIB between them, and replace each state number by their position in the ranking.

In [14]:
def replace(arr, old_values, new_values):
    result = arr.copy()

    for old_val, new_val in zip(old_values, new_values):
        result[result == old_val] = new_val

    return result

In [17]:
# Where put district of columbia ? super high PIB

new_rank_state = [27,46,20,34,1,16,23,42,0,4,9,38,40,5,19,30,
                 33,28,24,43,15,11,14,17,12,22,48,35,32,39,8,37,3,
                 12,45,7,29,25,6,44,26,47,18,2,31,50,13,10,41,21,49,51,52]
old_states_name =  [1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,
                    29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,66,72]

ranking_column = replace(data_load[:,0], old_states_name, new_rank_state)

Next 5 columns are useless, they are about interview date and index of patient
## COLUMNS TO DELETE 2 3 4 5 7 8 9 10 11 12 14 19 20 21 22 23

## POSSIBLE MEAN 13 24 25 26 27 28 29 33 37 59 60 62 63 75 77 78 79 80 81 82 83 84 85 86 89 90 92 93 98 110 111 112 113 114

The 6th column is DISPCODE, so if interview was complete or not, I change value to 1 if complete 0 if not.

## Functions to help conversions of differents answers in one single metric parameter : 

In [2]:
def IntoPounds(x):
    if x >= 9000 :
        return int((x - 9000) * 2.20462)
    else:
        return x 

def IntoInches(x):
    if x < 9000:          
        return np.floor(x/100)*12 + (x % 100)
    else: 
        return (x - 9000) * 0.393701

def WeekToMonth(x):
    x_str = str(x)
    if x_str[0] == "1":       
        return (4.25*int(x_str[-2:])).astype(int)
    else:
        return int(x_str[-2:])
    
def DayToMonth(x):
    x_str = str(x)
    if x_str[0] == "1":       
        return 30 *int(x_str[-2:])
    elif x_str[0] == "2":
        return (4.25*int(x_str[-2:])).astype(int)
    else 
        return int(x_str[-2:])
    
def DayToYear(x):
    x_str = str(x)
    if x_str[0] == "1":       
        return 365 *int(x_str[-2:])
    elif x_str[0] == "2":
        return 52*int(x_str[-2:])
    elif x_str[0] == "3":
        return 12 * int(x_str[-2:])
    else
        return int(x_str[-2:])

def HourToMinutes(x):
    x_str = str(x)
    return int(x_str[0])*60 + int(x_str[-2:])
    
#def DateType(x):
#    x_str = str(x)
#    year = x_str[-4:]
#    month = x_str[:2] if len(x) == 6 else x_str[0]
#    return np.datetime64(year + '-' + month)

In [3]:
data_load[:,6] = replace(data_load[:,6], [1100,1200], [1,0])
data_load[:,13] = replace(data_load[:,13], [0,1], [1,2])
data_load[:,24] = replace(data_load[:,24], [1,2,7,9], [0,1,np.nan,np.nan])
data_load[:,25] = replace(data_load[:,25], [77,99], [np.nan,np.nan])
data_load[:,26] = replace(data_load[:,26], [2,3,4,5,7,9], [0.75,0.5,0.25,0,np.nan,np.nan])

array_1 = [27,28,29]

for i in array_1 : 
    data_load[:,i] = replace(data_load[:,i], [88,77,99], [np.nan,np.nan,np.nan])

data_load[:,31] = replace(data_load[:,31], [3,7,9], [0,np.nan,np.nan])

array_2 = [30,32,34,35,36,38,39,40,41,42,43,44,45,46,47,48,53,54,55,56,57,61,64,65,66,67,68,69,70,71,72,73,74,87,95,96,100,103,104,107,108,116,117,118]
           
for i in array_2:
    data_load[:,i] = replace(data_load[:,i], [7,9], [np.nan,np.nan])
    

data_load[:,33] = replace(data_load[:,33], [1,2,3,4,7,8,9], [6,18,42,60,np.nan,120,np.nan])
data_load[:,37] = replace(data_load[:,37], [1,2,3,4,7,9], [6,18,42,60,np.nan,np.nan])
data_load[:,49] = replace(data_load[:,49], [98,99], [np.nan,np.nan])

array_3 = [51,52,58]

for i in array_3 : 
    data_load[:,i] = replace(data_load[:,i], [9], [np.nan])
    
data_load[:,59] = replace(data_load[:,59], [88,99], [0,np.nan])
data_load[:,60] = replace(data_load[:,60], [1,2,3,4,5,6,7,8,77,99] , [5,12.5,17.5,22.5,30,42.5,62.5,75,np.nan,np.nan])

array_4 = [62,63]

for i in array_4 :
    data_load[:,i] = replace(data_load[:,i], [7777,9999], [np.nan,np.nan])    
    data_load[:,i] = list(map(IntoPounds,(data_load[:, i])))

data_load[:,75] = replace(data_load[:,75],[1,2,3,4,5,6,7,8,77,99] , [15,60,135,270,1080,2070,3600,np.nan,np.nan,np.nan])
data_load[:,76] = replace(data_load[:,76],[3,7,9] ,[0,np.nan,np.nan])
data_load[:,77] = replace(data_load[:,77],[777,888,999] ,[np.nan,0,np.nan])
data_load[:,77] = list(map(WeekToMonth,(data_load[:, 77])))


array_5 = [78,80,88,91,98,119]

for i in array_5 :
    data_load[:,i] = replace(data_load[:,i], [77,99], [np.nan,np.nan])
    
data_load[:,79] = replace(data_load[:,79],[77,88,99] ,[np.nan,0,np.nan])

array_6 = [81,82,83,84,85,86]

for i in array_6 :
    data_load[:,i] = replace(data_load[:,i], [300,555,777,999],  [0,0,np.nan,np.nan])
    data_load[:,i] = list(map(DayToMonth,(data_load[:, i])))
    
array_7 = [89,90,92,93] 

for i in array_7 :
    data_load[:,i] = replace(data_load[:,i], [777,999],  [0,0,np.nan,np.nan])

data_load[:,89] = list(map(WeekToMonth,(data_load[:, 89])))
data_load[:,90] = list(map(HourToMinutes,(data_load[:, 90])))
data_load[:,92] = list(map(WeekToMonth,(data_load[:, 92])))

array_7 = [94,110,111] 

for i in array_8 :
    data_load[:,i] = replace(data_load[:,i], [777,888,999], [np.nan,0,np.nan])

data_load[:,94] = replace(data_load[:,94], [777,888,999], [np.nan,0,np.nan])
data_load[:,94] = list(map(WeekToMonth,(data_load[:, 94])))
data_load[:,97] = replace(data_load[:,97], [2,3,7,9], [0.5,0,np.nan,np.nan])
data_load[:,99] = replace(data_load[:,99], [2,3,4,5,7,8,9], [0.75,0.5,0.25,0,np.nan,np.nan,np.nan])
data_load[:,101] = replace(data_load[:,101], [777777, 999999],  [np.nan,np.nan])

#data_load[:,101] = list(map(DateType,(data_load[:, 101])))

data_load[:,105] = replace(data_load[:,105], [777777, 999999],  [np.nan,np.nan])
#data_load[:,105] = list(map(DateType,(data_load[:, 105])))

data_load[:,110] = list(map(DayToYear,(data_load[:, 110])))
data_load[:,111] = list(map(DayToYear,(data_load[:, 111])))

data_load[:,113] = replace(data_load[:,113],[77,88,98,99] ,[np.nan,0,np.nan,np.nan])
data_load[:,114] = replace(data_load[:,114],[77,88,99] ,[np.nan,0,np.nan])
data_load[:,115] = replace(data_load[:,114],[1,2,3,4,7,8,9] ,[15,180,540,720,np.nan,0,np.nan])



NameError: name 'replace' is not defined

9th to 13th column are bound together, they help getting the answer on the last column CELLFON3, so we can remove the 9th, 10th, 11th and 12th columns.
I create a new column to know if the survey is done on a cellphone or fixed phone.


Column 15,16,17 is super useful and tells us about the number of adults in the household, but they are not logic. Number of household where 1 adult < Number of household where 1 adult men

19th column to 24th bound together so only last one useful (24th)
We denote the value in the 24th column 0 if the person is a woman and doesnt have a fixed number to receive calls, 1 if she has one.
2 if the person is a man and doesnt have a fixed number to receive calls, 3 if he has one.


Column 25 is super interesting tells us the number of adults in the household and is similar to column 15 but seems more logic !
But the values are extremes we have to change them !


We clean the data by removing the nan values by the mean of the rest of the feature

In [ ]:
data = np.ones(data_load.shape)
stds = np.array([])
for i in range(data.shape[1]):
    d, std = standardize_clean(data_load[:, i])
    data[:, i] = d
    stds = np.append(stds, std)
print(stds.shape)

We further clean the data by removing the features where the variance is zero since they are constants for all samples

In [ ]:
indices = np.where(stds != 0)
data_var = data[:, indices]
data_var = np.squeeze(data_var, axis = 1)
print(data_var.shape)

We also remove the 8 first features as the appear weird in the task of predicting a heart attack

In [ ]:
data_cleaned = data_var[:, 9:]
print(data_cleaned.shape)

We then separe the data to train on 60% of the total and to test it on the remaining 40% 

In [ ]:
train_size = np.floor(data_cleaned.shape[0] * 0.6).astype(int)
data_cross = data_cleaned[:train_size, :]
pred_cross = pred[:train_size]
data_test = data_cleaned[train_size:, :]
pred_test = pred[train_size:]
print(f"Cross shape : {data_cross.shape} ; Test shape : {data_test.shape}")

Now we build our models for linear regression

In [ ]:
y, tx = build_model_data(data_cross, pred_cross)
y_test, tx_test = build_model_data(data_test, pred_test)
print(f"The data has now {tx.shape[1]} features !")